<a href="https://colab.research.google.com/github/nutapol97/Python-for-DS-AI_Nutapol_T./blob/main/AdaBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from sklearn.datasets import make_moons
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
def train_test(X,Y,size):
  from numpy.random import default_rng
  rng = default_rng()
  SizeTrain=int((size*X.shape[0]))
  random_np=rng.choice(X.shape[0], size=SizeTrain, replace=False)
  X_test=X[random_np,:]
  y_test=Y[random_np,]
  Y_train=np.delete(Y, random_np,axis=0)
  X_train=np.delete(X, random_np,axis=0)
  Y_train=Y_train.reshape(Y_train.shape[0],)
  y_test=y_test.reshape(y_test.shape[0],)
  return X_train,X_test,Y_train,y_test

In [24]:
X, y = make_classification(n_samples=500)
y = np.where(y==0,-1,1) 
X_train, X_test, y_train, y_test = train_test(X, y, 0.3)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [25]:
class DecisionStump():
    def __init__(self):
        self.po = 1
        self.feature_index = None
        self.threshold = None
        self.alpha = None
class AdaBoost():
    def __init__(self, S=5, eta=0.5):
        self.S = S
        self.eta = eta
    def fit(self, X, y):
      self.clfs = []
      m, n = X.shape
      W = np.full(m, 1/m)
      for _ in range(self.S):
          clf = DecisionStump()
          min_err = np.inf
      for feature in range(n):
                f_vals = np.sort(np.unique(X[:, feature]))
                th = (f_vals[:-1] + f_vals[1:])/2
                for threshold in th:
                    for po in [1, -1]:
                        yhat = np.ones(len(y)) 
                        yhat[po * X[:, feature] < po * threshold] = -1  
                        err = W[(yhat != y)].sum()
                        if err < min_err:
                            clf.po = po
                            clf.threshold = threshold
                            clf.feature_index = feature
                            min_err = err
      eps = 1e-10 
      clf.alpha = self.eta * (np.log ((1 - min_err) / (min_err + eps)))            
      W = W * np.exp(-clf.alpha * y * yhat)
      W = W / sum (W)
      self.clfs.append(clf)

    def predict(self, X):
        m, n = X.shape
        yhat = np.zeros(m)
        for clf in self.clfs:
            pred = np.ones(m) 
            pred[clf.po * X[:, clf.feature_index] < clf.po * clf.threshold] = -1  
            yhat += clf.alpha * pred
        return np.sign(yhat)          


model = AdaBoost(S=20)
model.fit(X_train, y_train)
yhat = model.predict(X_test)
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

          -1       0.89      0.81      0.85        72
           1       0.84      0.91      0.87        78

    accuracy                           0.86       150
   macro avg       0.86      0.86      0.86       150
weighted avg       0.86      0.86      0.86       150

